In [3]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

from keras.models import model_from_json
from skimage.transform import resize

Using TensorFlow backend.


In [6]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    age = int(ds.PatientAge)
    if age < 1 or age > 100:
        print("Age out of range")
        return None
    if ds.PatientPosition not in ['PA', 'AP']:
        print("Patient position not identified")
        return None
    if ds.BodyPartExamined not in ['CHEST', 'chest']:
        print("Patient body part not as per training")
        return None
    if ds.Modality != 'DX':
        print("Modality not in training")
        return None
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    img = img / 255
    img = resize(img, (img_size[1], img_size[2]), anti_aliasing=False)
    
    proc_img = img.reshape((1, img_size[1], img_size[2], 1))
    
    proc_img = np.repeat(proc_img, img_size[3], axis=3)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred = model.predict(img)
    prediction = pred > thresh
    
    # batch size: 1, single output
    return prediction[0][0]
#     result = model.predict(img)  
#     predict=result[0]
#     prediction='Not a pneumonia'
#     if(predict>thresh):
#         prediction='Pneumonia'    
#     return prediction 

In [7]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' #path to saved model
weight_path = 'xray_class_my_model.best.hdf5'#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = -1 # loads the mean image value they used during training preprocessing
img_std = -1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.3 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    print
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
True
Load file test2.dcm ...
True
Load file test3.dcm ...
True
Load file test4.dcm ...
Patient body part not as per training
Load file test5.dcm ...
Modality not in training
Load file test6.dcm ...
Patient position not identified
